In [1]:
import pandas as pd
import numpy as np
import re
import os
import string
import fn_module
from fn_module import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yukti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_file(directory = "CSE508_Winter2023_Dataset_Processed_new"):
    data = {}
    for filename in os.listdir(directory):    
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as f:
            file_contents = f.read()
        data[filename] = file_contents
    return data

data = read_file()
print("File count = ",len(data))

keys = list(data.keys())

s = "";
df = []
for key in data:
    df.append(data[key])
    


File count =  1400


In [3]:
from collections import defaultdict

def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    
    
    for doc_id, document in enumerate(documents):
        string1 = document.split()
        for word in range(len(string1)-1):
            bi_string = string1[word:word+2]
            bi = " ".join(bi_string)
            inverted_index[bi].append(doc_id)
    
    return inverted_index

ind = create_inverted_index(df)
print(ind);


defaultdict(<class 'list'>, {'experimental investigation': [0, 28, 83, 83, 169, 172, 175, 188, 244, 250, 371, 371, 422, 441, 441, 496, 504, 521, 551, 568, 634, 635, 661, 693, 712, 765, 771, 800, 815, 835, 843, 857, 886, 931, 985, 995, 1018, 1038, 1073, 1091, 1096, 1097, 1155, 1155, 1158, 1160, 1204, 1219, 1224, 1226, 1229, 1230, 1337, 1363], 'investigation aerodynamics': [0], 'aerodynamics wing': [0], 'wing slipstream': [0], 'slipstream experimental': [0, 483], 'experimental study': [0, 73, 255, 255, 333, 419, 463, 543, 548, 759, 771, 800, 846, 910, 1018, 1166, 1263], 'study wing': [0], 'wing propeller': [0], 'propeller slipstream': [0, 452, 452, 452, 1063, 1093, 1163], 'slipstream made': [0], 'made order': [0, 221], 'order determine': [0, 248, 276, 353, 730, 903, 984, 1318, 1392], 'determine spanwise': [0, 793], 'spanwise distribution': [0, 432, 673, 677, 1288], 'distribution lift': [0, 560, 670, 697], 'lift increase': [0, 162], 'increase due': [0], 'due slipstream': [0], 'slipstream 

In [4]:
import pickle
from collections import defaultdict

inverted_index = create_inverted_index(df)

def save_inverted_index(inverted_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(inverted_index, f)
        
def load_inverted_index(file_path):
    with open(file_path, 'rb') as f:
        inverted_index = pickle.load(f)
        
    return inverted_index

In [5]:
from collections import defaultdict


pos_index = defaultdict(list)
for no_1, doc in enumerate(df):
    conv_lower = doc.lower()
    t = conv_lower.split()
    for no_2, token_no in enumerate(t):
        if token_no not in pos_index:
            pos_index[token_no] = defaultdict(list)
        if no_2 not in pos_index[token_no]:
            pos_index[token_no][no_1] = []
        pos_index[token_no][no_1].append(no_2)
        
# print(pos_index)

In [6]:
import pickle 
def save_positional_index(pos_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(pos_index, f)
        
def load_positional_index(file_path):
    with open(file_path, 'rb') as f:
        pos_index = pickle.load(f)
    return pos_index

In [19]:
def and_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
    return final_set

def or_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
        
    while(ptr_2 < len_2):
        final_set.add(s_2[ptr_2])
        ptr_2 += 1
        
    result = sorted(final_set)
        
    return result, count_comparisons

def ornot_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
        
        
    result = sorted(final_set)
        
    return result, count_comparisons

def andnot_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
    return final_set, count_comparisons


In [20]:
print('T1 AND T2:', and_operation(inverted_index['flat plate'], inverted_index['leading edge']))


print('T1 AND T2:', or_operation(inverted_index['made order'], inverted_index['order determine']))

print('T1 AND T2:', ornot_operation(inverted_index['order determine'], inverted_index['made order']))


print('T1 AND T2:', andnot_operation(inverted_index['flat plate'], inverted_index['leading edge']))


T1 AND T2: {1, 1156, 7, 8, 787, 1299, 24, 154, 1197, 303, 306, 307, 309, 568, 570, 970, 333, 210, 1106, 1369}
T1 AND T2: ([0, 221, 248, 276, 353, 730, 903, 984, 1318, 1392], 2)
T1 AND T2: ([248, 276, 353, 730, 903, 984, 1318, 1392], 2)
T1 AND T2: ({513, 899, 900, 901, 520, 525, 654, 528, 145, 1168, 787, 1299, 791, 24, 25, 154, 283, 792, 793, 795, 543, 672, 796, 546, 419, 674, 932, 1319, 682, 1197, 438, 439, 568, 314, 571, 190, 192, 706, 1228, 333, 592, 210, 1106, 599, 600, 221, 1250, 104, 624, 1266, 756, 1271, 632, 633}, 252)


In [30]:

t = int(input())
while(t>0):
    t -= 1
    sentence = input()
    trans_sentence = fn_module.preprocess(sentence)
#     print(trans_sentence)
    
    bi_words = []
    for word in range(len(trans_sentence)-1):
        string = trans_sentence[word] +" "+ trans_sentence[word+1]
        bi_words.append(string)
        
    print(bi_words)
    
    x = len(bi_words)
    res = []
    ptr = 2
    temp = []
    temp.append(and_operation(inverted_index[bi_words[0]], inverted_index[bi_words[1]]))

    while(ptr < len(bi_words)-1):
        res.append(and_operation(temp, inverted_index[bi_words[ptr+1]]))
        print(res)
        ptr += 1

    print(res)
    

1
investigation aerodynamics wing slipstream
['investigation aerodynamics', 'aerodynamics wing', 'wing slipstream']
[]
